In [ ]:
# importing required libraries

import requests
import configparser
from pymongo import MongoClient
from settings import CHANNEL_ID, DB_NAME, VIDEO_COLLECTION # importing details from settings.py

In [ ]:
#  Configuration settings

config = configparser.ConfigParser()
config.read('credentials.ini')

# mongodb setupp

client = MongoClient(config['MONGODB']['uri'])
db = client[DB_NAME]
collection = db[VIDEO_COLLECTION]

URL = config['YOUTUBE']['url']

In [ ]:
# Making API request

# API parmeters

params = {
    'key': config['YOUTUBE']['api_key'],
    'channelId': CHANNEL_ID,
    'part': 'snippet,id', # this will fetch video IDs
    'order': 'date',
    'maxResults': 50,  # Max allowed by Youtbe API
    'type': 'video',
    'publishedAfter': '2020-01-01T00:00:00Z'  # Adding date filter to narrow results
}

response = requests.get(URL, params=params) # making the first API request
data = response.json() # converting API response into a jason file

In [ ]:
# Verify response

print(f"API returned {len(data['items'])} videos")
print(f"Quota status: {response.headers.get('x-ratelimit-remaining')}")

if 'items' in data:
    collection.insert_many(data['items'])

API returned 50 videos
Quota status: None


In [ ]:
# checking number of documents in mongodb collection

collection.count_documents({})

448

In [ ]:
# After initial request. Using next page token to get more videos from other pages

'''
This handles the pagination issue

'''

while 'nextPageToken' in data:
    params['pageToken'] = data['nextPageToken']
    response = requests.get("https://www.googleapis.com/youtube/v3/search", params=params)
    data = response.json()
    collection.insert_many(data['items']) # inserting into Mongodb

In [25]:
print(f"Total inserted: {len(data)} videos")

Total inserted: 6 videos


In [ ]:
# checking total number of documents in mongodb collection

collection.count_documents({})

742